<a href="https://colab.research.google.com/github/ankur334/pyspark-practice/blob/tricky-pyspark-problems/solutions/tricky_question2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=1cb3df77131ef27e699be912315b263c4372d2bfd3a88395a5211929778f98e8
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


#**Ques1**
##**Word Count Program**

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col
from pyspark.sql.functions import explode
from pyspark.sql.functions import lower
from pyspark.sql.functions import regexp_extract

# Create a Spark session

spark = SparkSession.builder.appName("WordCountData").getOrCreate()

df = spark.read.text('file:///content/sample_data/sample_data.txt')  #change path as per your system

#print(df.show(truncate=False))

# +--------------------+
# |               value|
# +--------------------+
# |Professor Amelia ...|
# |                    |
# |renowned historia...|
# |                    |
# |                    |
# | The last documen...|
# |                    |
# |Determined, Ameli...|
# |                    |
# |Title: The Curiou...|
# |                    |
# |Posting Date: Mar...|
# |                    |
# |   Language: English|
# |                    |
# |Character set enc...|
# +--------------------+

lines = df.select(split(col("value"), " ").alias("line"))
#print(lines.collect())
###[Row(line=['Professor', 'Amelia', 'Moore,']), Row(line=['']) ......]

words = lines.select(explode(col("line")).alias("word"))
# Cleaning : lowering and removing punctuations

words_lower = words.select(lower(col("word")).alias("word_lower"))
words_clean = words_lower.select(regexp_extract(col("word_lower"), "[a-z]+", 0).alias("word"))

#print(words_clean.collect())
###[Row(word='professor'), Row(word='amelia'), Row(word='moore'), Row(word=''),....]

words_nonnull = words_clean.filter(col("word") != "")
groups = words_nonnull.groupby(col("word")).count()
groups.orderBy("count", ascending=False).show(10)



+-----------+-----+
|       word|count|
+-----------+-----+
|        the|   11|
|          a|    7|
|         of|    5|
|     amelia|    4|
|bartholomew|    3|
| manuscript|    3|
|        was|    2|
|    captain|    2|
|         on|    2|
|     within|    2|
+-----------+-----+
only showing top 10 rows

